In [ ]:
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score
)
data = pd.read_csv("Dataset-2-New-1")

X = data.drop("Class", axis=1)
y = data["Class"]

# Normal transactions
normal = np.random.normal(loc=0, scale=1, size=(int(n_samples * 0.97), n_features))

# Fraud transactions (different distribution)
fraud = np.random.normal(loc=2, scale=1.5, size=(int(n_samples * 0.03), n_features))

X = np.vstack((normal, fraud))
y = np.hstack((
    np.zeros(len(normal)),
    np.ones(len(fraud))
))

columns = [f"feature_{i}" for i in range(n_features)]
data = pd.DataFrame(X, columns=columns)
data["Class"] = y

# -------- 2. Split Features & Target --------
X = data.drop("Class", axis=1)
y = data["Class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -------- 3. Feature Scaling --------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# -------- 4. Handle Class Imbalance --------
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

# -------- 5. XGBoost Model --------
model = XGBClassifier(
    n_estimators=250,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42
)

# -------- 6. Train --------
model.fit(X_train, y_train)

# -------- 7. Evaluate --------
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_prob))

# -------- 8. Real-Time Prediction Function --------
def predict_transaction(transaction):
    transaction = scaler.transform([transaction])
    prob = model.predict_proba(transaction)[0][1]

    if prob > 0.5:
        return "⚠️ Fraud Detected", round(prob, 4)
    else:
        return "✅ Legit Transaction", round(prob, 4)

# -------- 9. Demo --------
sample = X_test[0]
result, probability = predict_transaction(sample)

print("\nSample Transaction Result:")
print(result, "| Probability:", probability)




Confusion Matrix:
[[1938    2]
 [   2   58]]

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1940
         1.0       0.97      0.97      0.97        60

    accuracy                           1.00      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       1.00      1.00      1.00      2000

ROC-AUC Score: 0.9997422680412371

Sample Transaction Result:
✅ Legit Transaction | Probability: 1e-04


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
